## Heat transfer for heat sink

In [19]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie, GLMakie, LaTeXStrings, Makie.GeometryBasics

In [20]:
push!(LOAD_PATH, "utilities")
include("utilities/plot_design.jl")
include("utilities/OC.jl")
include("utilities/SIMP_Inter.jl")
include("utilities/filters.jl")
push!(LOAD_PATH, "FD_check")
include("FD_check/run_finite_difference_check.jl")

run_finite_difference_check (generic function with 2 methods)

## 1. Problem
### Let's consider the heat sink as follows:
<img src="imagines/heat_sink.png" style="width:400px;height:350px"/>

The dimensions of the cubic heat sink are $L \times L \times L$ . 

This sink is under a heat flux ($q_{in}$) at the bottom face with $y=0$. The top face is prescribed a temperature of $T = 0 \ ^\circ\mathrm{C}$
 load at $y=L$. The body heat sourse is $s$.


The weak form of this problem is given as follows:

Find $\mathbf{u}$ such that for all test functions $\mathbf{v}$:

$$
a(\mathbf{u}, \mathbf{v}) = l(\mathbf{v}),
$$

where:

$$
a(u,v) = \int_{\Omega} k \nabla v \cdot \nabla u \, d\Omega, 
\quad 
l(v) = -\int_{\Gamma_q} q_{\text{in}} v\, d\Gamma + \int_{\Omega} s v\, d\Omega.
$$

## Inputs

In [23]:
# === Geometry ===
L = 20  # Length of heat sink base
W = 20  # Width and Height
domain = (0, L, 0, W, 0, W)
partition = (5, 5, 5)
model = CartesianDiscreteModel(domain, partition)

Bottom = [1, 2, 5, 6, 9, 11, 17, 18, 23]
Top = [3,4,7,8,10,12,19,20,24]
diff_ = setdiff(1:26, Bottom)

# === Tags for boundary conditions ===
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "bottom", Bottom) # example indices for corners/edges
add_tag_from_tags!(labels, "top", Top) # Top
# === Tag elements at the center of the left face ===
# writevtk(model, "HeatSink")
# === Integration ===
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
Γ_flux = BoundaryTriangulation(model, tags = "bottom")
dΓ_flux = Measure(Γ_flux, degree)

# === FE space ===
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)

Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags=["top"])
T₀(x) = VectorValue(0)  # Celsius
Uₕ = TrialFESpace(Vₕ, [T₀])  # Essential BC: T = 0°C on the Dirichlet face



TrialFESpace()

In [4]:
# === Problem data ===
k = 5.0             # W/C
q_in = 20.0          # W/m (heat flux)
s = 6               # W/m^2 Body source term (set to non-zero if needed)

# === Weak form ===
a(u,v) = ∫( k * ∇(v) ⋅ ∇(u) ) * dΩ
l(v) = ∫( -1*q_in * v ) * dΓ_flux + ∫( s * v ) * dΩ

# === Solve ===
op = AffineFEOperator(a, l, Uₕ, Vₕ)
Th = solve(op)

# writevtk(Ω, "HeatSink_T", cellfields = ["T" => Th])

SingleFieldFEFunction():
 num_cells: 125
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 12915414997105604600

## Thermal compliance

In [5]:
T = get_free_dof_values(Th) # displacement vector
# Method 1
C1 = sum(∫( k * ∇(Th) ⋅ ∇(Th) ) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫( -1*q_in * Th ) * dΓ_flux + ∫( s * Th ) * dΩ) # Compliance
println("Compliance 2: ", C2)
# Method 3
u = get_free_dof_values(Th) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(u, K * u)
println("Compliance 3: ", C3)

Compliance 1: 4.403199999999991e6
Compliance 2: 4.4031999999999935e6
Compliance 3: 4.403199999999994e6


In [ ]:
num_ele = num_cells(Ω) # Number of elements
volfrac = 0.4
p = 3
max_iter = 50
filter_radius = 2.5
# === Geometry ===
L = 20  # Length of heat sink base
W = 20  # Width and Height
domain = (0, L, 0, W, 0, W)
partition = (5, 5, 5)
model = CartesianDiscreteModel(domain, partition)
Bottom = [1, 2, 5, 6, 9, 11, 17, 18, 23]
Top = [3,4,7,8,10,12,19,20,24]
diff_ = setdiff(1:26, Bottom)
# === Tags for boundary conditions ===
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "bottom", Bottom) # example indices for corners/edges
add_tag_from_tags!(labels, "top", Top) # Top
# === Tag elements at the center of the left face ===
# writevtk(model, "HeatSink")
# === Integration ===
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
Γ_flux = BoundaryTriangulation(model, tags = "bottom")
dΓ_flux = Measure(Γ_flux, degree)
# === FE space ===
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)

Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags=["top"])
T₀(x) = VectorValue(0)  # Celsius
Uₕ = TrialFESpace(Vₕ, [T₀])  # Essential BC: T = 0°C on the Dirichlet face
# ------------
H = filters(filter_radius, Ω);

In [17]:
fd_step_size = 1e-6
dv = rand(num_ele)
type_FEA = "Thermal"
figs = run_finite_difference_check(type_FEA, dv, fd_step_size, H, p, k, q_in, s, Ω, dΩ, dΓ_flux, Uₕ, Vₕ)
display(figs)

Computing finite difference sensitivities of cost...
Max. ABSOLUTE error is:  -1.21125e+01
It occurs at variable of  110
Max. RELATIVE error is:  -5.14832e-07
It occurs at variable of  110


GLMakie.Screen(...)

In [ ]:
for iter = 1:max_iter
    if iter == 1
        ρ_new = 0.5 * ones(num_ele)
    end
    ρ_old = copy(ρ_new)
    ρ_new_fil = H * ρ_new # Filter
    # SIMP
    ρ_SIMP, dρ_dp = SIMP_Inter(ρ_new_fil, p, Ω)
    # ===FE-ANALYSIS
    a(u,v) = ∫(ρ_SIMP * k * ∇(v) ⋅ ∇(u) ) * dΩ
    l(v) = ∫( -1*q_in * v ) * dΓ_flux + ∫( s * v ) * dΩ
    # Solution of the FE problem
    op = AffineFEOperator(a, l, Uₕ, Vₕ)
    Th = solve(op)
    C = sum(∫( ρ_SIMP *k * ∇(Th) ⋅ ∇(Th) ) * dΩ) # Compliance
    dC_fil = get_contribution(∫( -dρ_dp*k * ∇(Th) ⋅ ∇(Th) ) * dΩ, Ω) # Sensitivity of compliance w.r.t rho
    dC = transpose(H) * (dC_fil[:])

    ρ_new = OC(ρ_new, volfrac, dC, num_ele)
    change_obj = maximum(abs.(ρ_new .- ρ_old))
    # figs = plot_design(ρ_new, cell_coords, iter, L, W)

    # display(figs)

    if iter % 10 == 0
        println("It. ", iter, ", Obj = ", @sprintf("%.5e", C), ", Vol.: ", @sprintf("%.5e", sum(ρ_new) / num_ele), ", ch.: ", @sprintf("%.5e", change_obj))
    end

    if iter == max_iter
        writevtk(Ω, "Heat_Sink_TOP", cellfields = ["rho" => ρ_SIMP])
    end

end